In [75]:
import os
import gzip
import csv

import pandas as pd
import numpy as np

In [1]:
def generate_dirpaths(dirname='opensky-network.org'):
    """Yields the filenames that should be in a given directory.
    
    dirname: string directory name
    
    yields: directories under dirname and files they contain
    """
    for dirpath, dirnames, filenames in os.walk(dirname):
        if len(dirnames) == 0:
            yield dirpath, filenames

In [12]:
def download_datafiles(date, dirname='opensky-network.org/datasets/states'):
    """Download
    
    date: string 'YYYY-MM-DD' format
    
    """
    for hour in range(24):
        hour = '%0.2d' % hour
        
        filename = 'states_%s-%s.csv.tar' % (date, hour)

        cmd = 'wget -r https://%s/%s/%s/%s' % (dirname, date, hour, filename)
        print(cmd)

In [14]:
date = '2018-02-05'
download_datafiles(date)

wget -r https://opensky-network.org/datasets/states/2018-02-05/00/states_2018-02-05-00.csv.tar
wget -r https://opensky-network.org/datasets/states/2018-02-05/01/states_2018-02-05-01.csv.tar
wget -r https://opensky-network.org/datasets/states/2018-02-05/02/states_2018-02-05-02.csv.tar
wget -r https://opensky-network.org/datasets/states/2018-02-05/03/states_2018-02-05-03.csv.tar
wget -r https://opensky-network.org/datasets/states/2018-02-05/04/states_2018-02-05-04.csv.tar
wget -r https://opensky-network.org/datasets/states/2018-02-05/05/states_2018-02-05-05.csv.tar
wget -r https://opensky-network.org/datasets/states/2018-02-05/06/states_2018-02-05-06.csv.tar
wget -r https://opensky-network.org/datasets/states/2018-02-05/07/states_2018-02-05-07.csv.tar
wget -r https://opensky-network.org/datasets/states/2018-02-05/08/states_2018-02-05-08.csv.tar
wget -r https://opensky-network.org/datasets/states/2018-02-05/09/states_2018-02-05-09.csv.tar
wget -r https://opensky-network.org/datasets/state

In [4]:
import tarfile

In [5]:
filename = 'opensky-network.org/datasets/states/2017-07-24/00/states_2017-07-24-00.csv.tar'

In [6]:
tf = tarfile.open(filename)

In [7]:
tf.list()

?rw-r--r-- root/root   71408154 2017-08-09 11:53:44 states_2017-07-24-00.csv.gz 
?rw-r--r-- root/root      14973 2017-08-24 10:41:59 LICENSE.txt 
?rw-r--r-- root/root       7074 2017-09-05 05:32:47 README.txt 


In [8]:
for member in tf.getmembers():
    if member.name.startswith('states'):
        f = tf.extractfile(member)
        g = gzip.open(f)
        line = g.readline()
        print(line)

b'time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,alert,spi,squawk,baroaltitude,geoaltitude,lastposupdate,lastcontact\n'


In [27]:
def open_tarfile(filename):
    tf = tarfile.open(filename)
    for member in tf.getmembers():
        if member.name.endswith('.csv.gz'):
            f = tf.extractfile(member)
            g = gzip.open(f, mode='rt')
            reader = csv.reader(g)
            return reader

In [65]:
def open_tarfile_chunks(filename):
    tf = tarfile.open(filename)
    for member in tf.getmembers():
        if member.name.endswith('.csv.gz'):
            f = tf.extractfile(member)
            reader = pd.read_csv(f, compression='gzip', chunksize=10000)
            return reader

In [67]:
reader = open_tarfile_chunks(filename)
reader

In [68]:
chunk = next(reader)
chunk.head()

,time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,alert,spi,squawk,baroaltitude,geoaltitude,lastposupdate,lastcontact
0,1500854400,406735,49.883788,7.112167,225.056885,303.436890,0.00000,EXS938,False,False,False,4761.0,11887.20,NaN,1.500854e+09,1.500854e+09
1,1500854400,406900,52.228678,-1.065125,148.049082,6.384164,-11.70432,EXS632,False,False,False,7454.0,4396.74,4404.36,1.500854e+09,1.500854e+09
2,1500854400,a009a4,40.924072,-73.555542,187.768536,340.472890,6.82752,AAL21,False,False,False,7760.0,3634.74,3749.04,1.500854e+09,1.500854e+09
3,1500854400,4b8e05,50.838547,5.594723,267.219727,101.546631,0.00000,PGT1174,False,False,False,4767.0,10058.40,10126.98,1.500854e+09,1.500854e+09
4,1500854400,3c0f31,38.245468,-77.052612,216.458082,240.998301,0.32512,BMW54A,False,False,False,745.0,13106.40,13784.58,1.500854e+09,1.500854e+09


In [86]:
from math import radians, cos, sin, asin, sqrt

def haversine(*args):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    
    From https://stackoverflow.com/questions/4913349/haversine-formula
    -in-python-bearing-and-distance-between-two-gps-points
    """
    # convert decimal degrees to radians 
    lat1, lon1, lat2, lon2 = np.deg2rad(args)

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [87]:
reader = open_tarfile(filename)
header = next(reader)
header

['time',
 'icao24',
 'lat',
 'lon',
 'velocity',
 'heading',
 'vertrate',
 'callsign',
 'onground',
 'alert',
 'spi',
 'squawk',
 'baroaltitude',
 'geoaltitude',
 'lastposupdate',
 'lastcontact']

In [88]:
logan = 42.3656, -71.0096

In [89]:
res = {}

for i, line in enumerate(reader):
    time, icao24 = line[:2]
    try:
        lat = float(line[2])
        lon = float(line[3])
    except ValueError:
        continue
        #print(line)
        
    d = haversine(lat, lon, *logan)
    if d < 10:
        res.setdefault(icao24, []).append(time)
        #print(line)
        
    if i > 10000000:
        break

In [90]:
len(res)

24

In [91]:
res

{'3949e7': ['1500854790', '1500854800', '1500854810'],
 '396440': ['1500854430',
  '1500854440',
  '1500854450',
  '1500854460',
  '1500854470',
  '1500854480',
  '1500854490',
  '1500854500',
  '1500854510',
  '1500854520',
  '1500854530',
  '1500854540',
  '1500854550',
  '1500854560',
  '1500854570',
  '1500854580',
  '1500854590',
  '1500854600',
  '1500854610',
  '1500854620',
  '1500854630',
  '1500854640',
  '1500854650',
  '1500854660',
  '1500854670',
  '1500854680',
  '1500854690',
  '1500854700',
  '1500854710',
  '1500854720',
  '1500854730',
  '1500854740',
  '1500854750',
  '1500854760'],
 '40067f': ['1500854610',
  '1500854620',
  '1500854630',
  '1500854640',
  '1500854650',
  '1500854660',
  '1500854670'],
 '406b48': ['1500855660',
  '1500855670',
  '1500855680',
  '1500855690',
  '1500855700',
  '1500855710',
  '1500855720'],
 '896185': ['1500854560', '1500854570', '1500854580'],
 'a0ec27': ['1500856270',
  '1500856280',
  '1500856290',
  '1500856300',
  '1500856310',

In [92]:
import redis

r = redis.Redis(
    host='localhost',
    port=6379)

In [93]:
r.set('foo', 'bar')

True

In [94]:
r.get('foo')

b'bar'

In [113]:
def process_file(reader, redis, nrows=1e9):
    for i, line in enumerate(reader):
        time, icao24 = line[:2]
        try:
            lat = float(line[2])
            lon = float(line[3])
        except ValueError:
            continue
            #print(line)
        
        d = haversine(lat, lon, *logan)
        if d < 10:
            key = 'icao24:' + icao24
            r.hsetnx(key, time, line)
            #print(line)
        
        if i > nrows:
            break

In [114]:
reader = open_tarfile(filename)
header = next(reader)

%time process_file(reader, r)

CPU times: user 31.5 s, sys: 40 ms, total: 31.5 s
Wall time: 31.5 s


In [115]:
keys = r.keys()
keys

[b'icao24:a65c95',
 b'icao24:a662a5',
 b'icao24:a15703',
 b'icao24:ad1560',
 b'icao24:a2cb14',
 b'icao24:a189a7',
 b'icao24:a8dee0',
 b'icao24:a7c8ec',
 b'icao24:a3deca',
 b'icao24:ab1628',
 b'icao24:a96c96',
 b'icao24:3949e7',
 b'icao24:a83876',
 b'icao24:a23efc',
 b'icao24:acf5fe',
 b'icao24:896185',
 b'icao24:ab4ae9',
 b'icao24:aa56b3',
 b'icao24:396440',
 b'icao24:ab4fd4',
 b'icao24:ab81b5',
 b'icao24:406b48',
 b'icao24:a0ec27',
 b'icao24:40067f']

In [118]:
r.hgetall('icao24:a65c95')

{b'1500857690': b"['1500857690', 'a65c95', '42.2818613861', '-71.0453264103', '92.1242666789', '20.2564621745', '-4.55168', 'NKS414  ', 'False', 'False', 'False', '3672', '441.96', '464.82', '1500857688.7', '1500857689.58']",
 b'1500857700': b"['1500857700', 'a65c95', '42.2890769829', '-71.0418133403', '79.5593924753', '19.2550777893', '-9.7536', 'NKS414  ', 'False', 'False', 'False', '3672', '411.48', '434.34', '1500857699.08', '1500857699.9']",
 b'1500857710': b"['1500857710', 'a65c95', '42.2952575684', '-71.0388461026', '78.104053042', '19.6284382469', '-3.57632', 'NKS414  ', 'False', 'False', 'False', '3672', '342.9', '365.76', '1500857708.25', '1500857709.94']",
 b'1500857720': b"['1500857720', 'a65c95', '42.2980041504', '-71.0375352339', '75.5094447212', '19.9164059938', '-2.60096', 'NKS414  ', 'False', 'False', 'False', '3672', '327.66', '350.52', '1500857712.36', '1500857715.91']",
 b'1500857730': b"['1500857730', 'a65c95', '42.3076047736', '-71.0328709802', '75.5094447212', '1

In [117]:
def delete_all(r):
    keys = r.keys()
    return r.delete(*keys)

In [112]:
r.keys()

[]

In [34]:
from datetime import datetime
import pytz

In [42]:
start = datetime(2017, 9, 3, 18, 0, 0)

eastern = pytz.timezone('US/Eastern')
loc_dt = eastern.localize(start)

fmt = '%Y-%m-%d %H:%M:%S %Z%z'
loc_dt.strftime(fmt)

'2017-09-03 18:00:00 EDT-0400'

In [43]:
start.timestamp()

1504476000.0

In [17]:
eastern = pytz.timezone('US/Eastern')

In [26]:
loc_dt = eastern.localize()
loc_dt

datetime.datetime(2017, 9, 3, 10, 0, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)

In [27]:
loc_dt.timestamp()

1504447200.0

In [28]:
loc_dt.astimezone(pytz.utc)

datetime.datetime(2017, 9, 3, 14, 0, tzinfo=<UTC>)

In [29]:
loc_dt.astimezone(pytz.utc).timestamp()

1504447200.0

In [44]:
import pandas as pd

In [52]:
df = pd.read_csv('data.csv', sep='|', skiprows=2)
df.columns

Index(['+------------+--------+-------------------+--------------------+-------------------+-------------------+--------------------+----------+----------+-------+-------+--------+-------------------+-------------------+----------------+----------------+------------+'], dtype='object')

In [66]:
from io import StringIO

In [170]:
def clean_csv(filename):
    res = StringIO()
    with open('data.csv') as f:
        for line in f:
            if line.startswith('|'):
                line = line.strip('|').replace(' ', '')
                res.write(line)
    res.seek(0)
    return res

In [183]:
from datetime import datetime

converters = {}
converters['time'] = datetime.fromtimestamp

In [185]:
res = clean_csv('data.csv')
df = pd.read_csv(res, sep='|')
df.columns

Index(['time', 'icao24', 'lat', 'lon', 'velocity', 'heading', 'vertrate',
       'callsign', 'onground', 'alert', 'spi', 'squawk', 'baroaltitude',
       'geoaltitude', 'lastposupdate', 'lastcontact', 'hour', 'Unnamed: 17'],
      dtype='object')

In [186]:
df.head()

,time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,alert,spi,squawk,baroaltitude,geoaltitude,lastposupdate,lastcontact,hour,Unnamed: 17
0,1504478054,a2a3b5,42.330002,-71.050644,287.578337,53.437590,0.32512,UAL915,False,False,False,566.0,11879.58,12252.96,1.504478e+09,1.504478e+09,1504476000,NaN
1,1504478055,a2a3b5,42.330788,-71.049287,287.885087,53.355353,0.32512,UAL915,False,False,False,566.0,11887.20,12252.96,1.504478e+09,1.504478e+09,1504476000,NaN
2,1504478056,a2a3b5,42.332324,-71.046476,287.885087,53.355353,0.32512,UAL915,False,False,False,566.0,11887.20,12252.96,1.504478e+09,1.504478e+09,1504476000,NaN
3,1504478057,a2a3b5,42.333860,-71.043666,287.885087,53.355353,0.00000,UAL915,False,False,False,566.0,11887.20,12252.96,1.504478e+09,1.504478e+09,1504476000,NaN
4,1504478058,a2a3b5,42.335071,-71.041430,287.885087,53.355353,0.00000,UAL915,False,False,False,566.0,11887.20,12252.96,1.504478e+09,1.504478e+09,1504476000,NaN


In [187]:
df.velocity.describe()

count    974.000000
mean     226.694084
std       71.282219
min       76.347719
25%      198.737195
50%      264.536789
75%      287.885087
max      303.838309
Name: velocity, dtype: float64

In [188]:
df.describe()

,time,lat,lon,velocity,heading,vertrate,squawk,baroaltitude,geoaltitude,lastposupdate,lastcontact,hour,Unnamed: 17
count,9.850000e+02,985.000000,985.000000,974.000000,974.000000,974.000000,915.000000,984.000000,720.000000,9.850000e+02,9.850000e+02,9.850000e+02,0.0
mean,1.504487e+09,42.354930,-70.998439,226.694084,111.676738,-4.261609,4575.113661,7399.174878,7972.213333,1.504487e+09,1.504487e+09,1.504485e+09,NaN
std,7.651054e+03,0.024748,0.039365,71.282219,75.372702,9.881351,2491.268177,3845.191727,4465.377551,7.601816e+03,7.647169e+03,7.751409e+03,NaN
min,1.504476e+09,42.330002,-71.069471,76.347719,46.989624,-16.906240,237.000000,449.580000,426.720000,1.504476e+09,1.504476e+09,1.504476e+09,NaN
25%,1.504479e+09,42.336700,-71.026986,198.737195,53.620369,-16.906240,2462.000000,5913.120000,1409.700000,1.504479e+09,1.504479e+09,1.504476e+09,NaN
50%,1.504484e+09,42.339203,-71.005575,264.536789,53.848007,-0.325120,5750.000000,7425.690000,10271.760000,1.504484e+09,1.504484e+09,1.504483e+09,NaN
75%,1.504495e+09,42.373535,-70.952773,287.885087,183.301866,0.325120,7154.000000,10668.000000,11597.640000,1.504495e+09,1.504495e+09,1.504494e+09,NaN
max,1.504496e+09,42.409880,-70.950581,303.838309,302.619243,21.783040,7306.000000,11887.200000,12252.960000,1.504496e+09,1.504496e+09,1.504494e+09,NaN


In [189]:
for column in df.columns:
    print(column, df[column].dtype)

time int64
icao24 object
lat float64
lon float64
velocity float64
heading float64
vertrate float64
callsign object
onground bool
alert bool
spi bool
squawk float64
baroaltitude float64
geoaltitude float64
lastposupdate float64
lastcontact float64
hour int64
Unnamed: 17 float64


In [190]:
df.onground.value_counts()

False    984
True       1
Name: onground, dtype: int64

In [191]:
df.time

0      1504478054
1      1504478055
2      1504478056
3      1504478057
4      1504478058
5      1504478059
6      1504478060
7      1504478061
8      1504478062
9      1504478063
10     1504478064
11     1504478065
12     1504478066
13     1504478067
14     1504478068
15     1504478069
16     1504478070
17     1504478071
18     1504478072
19     1504478073
20     1504478074
21     1504478075
22     1504478076
23     1504478077
24     1504478078
25     1504478079
26     1504478080
27     1504478081
28     1504478082
29     1504478083
          ...    
955    1504478604
956    1504478605
957    1504478606
958    1504478607
959    1504478608
960    1504478609
961    1504480454
962    1504480455
963    1504480456
964    1504480457
965    1504480458
966    1504480459
967    1504480460
968    1504480461
969    1504480462
970    1504480463
971    1504480464
972    1504480465
973    1504480466
974    1504480467
975    1504480468
976    1504480469
977    1504480470
978    1504480471
979    150

In [192]:
grouped = df.groupby('icao24')
len(grouped)

27

In [193]:
for name, group in grouped:
    print(name, len(group))

06a0fb 28
3c4b2d 38
3c4b31 24
400615 25
400684 25
4006b2 8
4007f4 41
401000 45
40608f 5
479a4d 50
4843f3 32
4ca615 16
4ca842 22
738041 48
a05614 6
a12861 13
a2a3b5 38
a300b6 33
a4f3d9 300
a69549 15
a6cad3 7
a7debf 12
a8bc71 9
aa635c 24
aaf8f1 30
ab80f4 22
abf2c2 69


In [199]:
grouped.get_group('4006b2')

,time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,alert,spi,squawk,baroaltitude,geoaltitude,lastposupdate,lastcontact,hour,Unnamed: 17
452,1504482311,4006b2,42.398894,-71.068513,297.101988,54.300861,0.00000,BAW11J,False,False,False,7143.0,10668.0,10949.94,1.504482e+09,1.504482e+09,1504479600,NaN
453,1504482312,4006b2,42.400589,-71.065625,297.101988,54.300861,0.00000,BAW11J,False,False,False,7143.0,10668.0,10949.94,1.504482e+09,1.504482e+09,1504479600,NaN
454,1504482313,4006b2,42.402199,-71.062572,297.101988,54.300861,-0.32512,BAW11J,False,False,False,7143.0,10668.0,10949.94,1.504482e+09,1.504482e+09,1504479600,NaN
455,1504482314,4006b2,42.403656,-71.059570,297.101988,54.300861,-0.32512,BAW11J,False,False,False,7143.0,10668.0,10949.94,1.504482e+09,1.504482e+09,1504479600,NaN
456,1504482315,4006b2,42.405178,-71.056696,296.802089,54.381510,0.00000,BAW11J,False,False,False,7143.0,10668.0,10949.94,1.504482e+09,1.504482e+09,1504479600,NaN
457,1504482316,4006b2,42.406631,-71.053952,296.802089,54.381510,0.00000,BAW11J,False,False,False,7143.0,10668.0,10949.94,1.504482e+09,1.504482e+09,1504479600,NaN
458,1504482317,4006b2,42.408297,-71.050820,296.802089,54.381510,0.00000,BAW11J,False,False,False,7143.0,10668.0,10949.94,1.504482e+09,1.504482e+09,1504479600,NaN
459,1504482318,4006b2,42.409698,-71.047960,296.802089,54.381510,-0.32512,BAW11J,False,False,False,7143.0,10668.0,10949.94,1.504482e+09,1.504482e+09,1504479600,NaN


In [200]:
df[df.baroaltitude<1000]

,time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,alert,spi,squawk,baroaltitude,geoaltitude,lastposupdate,lastcontact,hour,Unnamed: 17
44,1504476652,a7debf,42.337555,-71.018184,NaN,NaN,NaN,NaN,False,False,False,NaN,579.12,NaN,1.504477e+09,1.504477e+09,1504476000,NaN
45,1504476653,a7debf,42.337555,-71.018184,NaN,NaN,NaN,NaN,False,False,False,NaN,586.74,NaN,1.504477e+09,1.504477e+09,1504476000,NaN
46,1504476654,a7debf,42.337555,-71.018184,90.490977,151.103584,6.50240,NaN,False,False,False,NaN,594.36,556.26,1.504477e+09,1.504477e+09,1504476000,NaN
47,1504476655,a7debf,42.337555,-71.018184,90.490977,151.103584,6.50240,NaN,False,False,False,NaN,594.36,556.26,1.504477e+09,1.504477e+09,1504476000,NaN
48,1504476656,a7debf,42.337555,-71.018184,90.490977,151.103584,6.50240,NaN,False,False,False,NaN,594.36,556.26,1.504477e+09,1.504477e+09,1504476000,NaN
49,1504476657,a7debf,42.337555,-71.018184,90.490977,151.103584,6.50240,NaN,False,False,False,NaN,617.22,556.26,1.504477e+09,1.504477e+09,1504476000,NaN
50,1504476658,a7debf,42.337555,-71.018184,90.490977,151.103584,6.50240,NaN,False,False,False,NaN,617.22,556.26,1.504477e+09,1.504477e+09,1504476000,NaN
51,1504476659,a7debf,42.337555,-71.018184,90.490977,151.103584,6.50240,NaN,False,False,False,NaN,617.22,556.26,1.504477e+09,1.504477e+09,1504476000,NaN
52,1504476660,a7debf,42.337555,-71.018184,90.490977,151.103584,6.50240,NaN,False,False,False,NaN,617.22,556.26,1.504477e+09,1.504477e+09,1504476000,NaN
53,1504476661,a7debf,42.337555,-71.018184,90.490977,151.103584,6.50240,NaN,False,False,False,NaN,617.22,556.26,1.504477e+09,1.504477e+09,1504476000,NaN
